In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import time
import random
import pandas as pd
import os

def get_trailer_and_views(movie_title, driver):
    try:
        search_url = f"https://www.youtube.com/results?search_query={movie_title}+trailer"
        print(f"Navegando a: {search_url}")
        driver.get(search_url)
        
        # Esperar a que los resultados de búsqueda se carguen
        print("Esperando a que los resultados de búsqueda se carguen...")
        WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '(//*[@id="thumbnail"])[1]')))
        print("Resultados de búsqueda cargados.")
        
        # Manejar el pop-up de cookies de YouTube si aparece
        handle_cookie_popup(driver)
        
        # Reducir el tiempo de espera después de aceptar el pop-up a 20 segundos
        print("Esperando 1 segundos adicionales para asegurar la carga completa...")
        time.sleep(1)
        
        # Obtener el conteo de vistas directamente desde los resultados de búsqueda
        print("Obteniendo el conteo de vistas desde los resultados de búsqueda...")
        view_count_xpath = '(//span[contains(@class, "inline-metadata-item") and contains(text(), "views")])[1]'
        view_count_element = WebDriverWait(driver, 40).until(
            EC.presence_of_element_located((By.XPATH, view_count_xpath))
        )
        view_count_text = view_count_element.text
        view_count_match = re.search(r'(\d+(\.\d+)?[MK]?|\d{1,3}(,\d{3})*)( views)', view_count_text)
        if view_count_match:
            view_count = view_count_match.group(1)
        else:
            view_count = "N/A"
        print(f"Vistas: {view_count}")
        
        return f"{view_count}"

    except Exception as e:
        print(f"Ocurrió un error: {e}")
        return "Error"

def handle_cookie_popup(driver):
    try:
        # Esperar a que el pop-up de consentimiento de cookies esté presente
        print("Esperando al pop-up de consentimiento de cookies...")
        consent_popup = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '//div[@id="content" and @class="style-scope ytd-consent-bump-v2-lightbox"]'))
        )
        print("Pop-up de consentimiento de cookies encontrado.")
        
        # Desplazarse dentro del pop-up para revelar los botones
        print("Desplazándose dentro del pop-up...")
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", consent_popup)
        print("Desplazamiento dentro del pop-up completo.")
        
        # Encontrar y hacer clic en el botón "Aceptar todo" usando el XPath proporcionado
        print("Ubicando el botón 'Aceptar todo'...")
        accept_button = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, '//button[@class="yt-spec-button-shape-next yt-spec-button-shape-next--filled yt-spec-button-shape-next--mono yt-spec-button-shape-next--size-m" and .//span[text()="Accept all"]]'))
        )
        print("Botón 'Aceptar todo' ubicado.")
        
        print("Haciendo clic en el botón 'Aceptar todo'...")
        accept_button.click()
        print("Botón 'Aceptar todo' clickeado.")
    except Exception as e:
        print("No se encontró el pop-up de consentimiento de cookies o no se pudo cerrar:", e)

def process_movie_titles(titles, start_index=0):
    # Ruta al ejecutable de ChromeDriver
    driver_path = 'C:/Program Files/chromedriver/chromedriver-win64/chromedriver.exe'  # Reemplazar con la ruta actual de chromedriver.exe

    # Configurar opciones de Chrome
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--start-maximized")
    
    # Agregar una lista de user agents
    user_agents = [
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'
    ]
    
    results = []
    
    for index, title in enumerate(titles[start_index:], start=start_index):
        user_agent = random.choice(user_agents)
        chrome_options.add_argument(f'user-agent={user_agent}')
        
        # Inicializar el WebDriver de Chrome usando Service y las opciones configuradas
        service = Service(driver_path)
        driver = webdriver.Chrome(service=service, options=chrome_options)
        
        try:
            view_count = get_trailer_and_views(title, driver)
            results.append((title, view_count))
            
            # Save progress every 2 titles
            if (index + 1) % 2 == 0:
                temp_df = pd.DataFrame(results, columns=['title', 'view_count'])
                temp_df.to_csv('temp_results_youtube.csv', index=False)
                print(f"Progreso guardado en el índice {index + 1}")
                
            time.sleep(random.uniform(5, 10))  # Delay between requests to mimic human behavior
            
        finally:
            driver.quit()
    
    return results

# Cargar el archivo CSV
input_file_path = r'C:/Users/sofia/Documents/Documentos/Master Computational Social Sciences/TFM/Master Thesis/Scrapping and Data Part 2/Youtube and Instagram Data/data_final_movies_2.csv'
df = pd.read_csv(input_file_path)

# Obtener los títulos de las películas
titles = df['title'].tolist()

# Check for previous progress
if os.path.exists('temp_results_youtube.csv'):
    temp_df = pd.read_csv('temp_results_youtube.csv')
    processed_titles = temp_df['title'].tolist()
    start_index = len(processed_titles)
else:
    start_index = 0

# Procesar los títulos de las películas
results = process_movie_titles(titles, start_index=start_index)

# Crear un DataFrame con los resultados
results_df = pd.DataFrame(results, columns=['title', 'view_count'])

# Guardar el DataFrame actualizado en un nuevo archivo CSV
output_file_path = r'C:/Users/sofia/Documents/Documentos/Master Computational Social Sciences/TFM/Master Thesis/Scrapping and Data Part 2/Youtube and Instagram Data/data_final_movies_with_views.csv'
results_df.to_csv(output_file_path, index=False)

print(f"Proceso completado. Resultados guardados en {output_file_path}")